In [95]:
import joblib
import numpy as np
import os
import random
from sklearn.linear_model import LinearRegression

In [96]:
model_path = "linreg.joblib"
c_path = "linreg.c"

Linear Regression Creation & Dump

In [97]:
X = np.array([[random.randint(0, 10), random.randint(0, 10)] for i in range(10)])
y = np.dot(X, np.array([random.randint(0, 10), random.randint(0, 10)])) + random.randint(0, 10)
reg = LinearRegression().fit(X, y)

joblib.dump(reg, model_path)

['linreg.joblib']

Model Loading



In [98]:
model = joblib.load(model_path)
model

LinearRegression()

Getting model's parameters

In [99]:
coeffs, biais = list(model.coef_), model.intercept_
coeff_len = len(model.coef_)
coeffs, biais, coeff_len

([5.999999999999998, 3.0000000000000004], 5.000000000000007, 2)

C Code generation

In [100]:
coeff_line = ""
for coeff in coeffs:
  if type(coeff) is int:
    coeff_line += f"{coeff}.f, "
  else:
    coeff_line += f"{coeff}f, "

if type(biais) is int:
  coeff_line += f"{biais}.f"
else:
  coeff_line += f"{biais}f"

c_code = f"""
#include <stdio.h>

float predict(float *features) {{
    float coeffs[] = {{{coeff_line}}};
    size_t coeff_len = {coeff_len};

    float result = 0;
    for (size_t i = 0; i < coeff_len; ++i) {{
        result += coeffs[i] * features[i];
    }}
    result += coeffs[coeff_len];

    return result;
}}

int main() {{
    float features[10] = {{10, 20}};
    printf("prediction: %f\\n", predict(features));
    return 0;
}}
"""
print(c_code)


#include <stdio.h>

float predict(float *features) {
    float coeffs[] = {5.999999999999998f, 3.0000000000000004f, 5.000000000000007f};
    size_t coeff_len = 2;

    float result = 0;
    for (size_t i = 0; i < coeff_len; ++i) {
        result += coeffs[i] * features[i];
    }
    result += coeffs[coeff_len];

    return result;
}

int main() {
    float features[10] = {10, 20};
    printf("prediction: %f\n", predict(features));
    return 0;
}



Saving the generated code

In [101]:
with open(c_path, "w") as f:
    f.write(c_code)

Prediction

In [102]:
!gcc linreg.c -o linreg

In [103]:
!./linreg

prediction: 125.000000


In [104]:
X_pred = [[10, 20]]
model.predict(X_pred[:coeff_len])

array([125.])

Our generated C program has the same prediction output than our python regression.